In [1]:
from model.clamp import CLaMPLite   

model = CLaMPLite.load_from_checkpoint('/home/yj/PycharmProjects/MIT/CLaMP/outputs/2023-10-21/03-47-05/epoch=6-step=1386.ckpt')
model.eval()

/home/yj/anaconda3/envs/llamat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CLaMPLite(
  (graph_encoder): CGCNN(
    (embedding): Linear(in_features=92, out_features=512, bias=True)
    (rbf): RBFExpansion()
    (convs): ModuleList(
      (0-2): 3 x CGConv(512, dim=80)
    )
    (conv_to_fc): Sequential(
      (0): Linear(in_features=512, out_features=128, bias=True)
      (1): Softplus(beta=1, threshold=20)
      (2): Linear(in_features=128, out_features=512, bias=True)
    )
  )
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
 

In [5]:
text = ["I am a student.", "I am a student."]
tokenizer = model.tokenizer
input = tokenizer(text, return_tensors="pt")
input

{'input_ids': tensor([[ 101,  453,  568,  105, 6128,  211,  102],
        [ 101,  453,  568,  105, 6128,  211,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [7]:
input.input_ids.shape, input.attention_mask.shape

(torch.Size([2, 7]), torch.Size([2, 7]))

In [6]:
model.text_encoder(input['input_ids'].to(model.device), input['attention_mask'].to(model.device))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4878, -0.0894, -0.4275,  ...,  0.1494, -0.3210,  1.4815],
         [-0.1404,  0.0760, -0.8578,  ...,  0.1239,  0.1869,  1.6717],
         [ 0.1667,  0.2629, -0.3731,  ...,  0.2571, -0.5682,  1.9967],
         ...,
         [-0.4988, -0.5652, -0.0850,  ...,  0.0650, -0.3613,  1.7268],
         [-0.4990, -0.1551, -0.3210,  ...,  0.2011, -0.3753,  1.5463],
         [-0.3354,  0.5048, -0.7920,  ..., -0.2029, -0.4281,  1.2430]],

        [[-0.4878, -0.0894, -0.4275,  ...,  0.1494, -0.3210,  1.4815],
         [-0.1404,  0.0760, -0.8578,  ...,  0.1239,  0.1869,  1.6717],
         [ 0.1667,  0.2629, -0.3731,  ...,  0.2571, -0.5682,  1.9967],
         ...,
         [-0.4988, -0.5652, -0.0850,  ...,  0.0650, -0.3613,  1.7268],
         [-0.4990, -0.1551, -0.3210,  ...,  0.2011, -0.3753,  1.5463],
         [-0.3354,  0.5048, -0.7920,  ..., -0.2029, -0.4281,  1.2430]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackw

In [8]:
import pandas as pd

df = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_test.parquet')
df

,atoms,space group symbol,crystal system,energy per atom,volume,formation energy per atom,pretty formula,energy above hull,band gap,density,...,gpt_explanation,reduced_formula,edge_index,edge_attr,num_nodes,node_feat,y,text,structure_question_list,composition_question_list
113701,"{'abc': [3.6269, 3.6269, 3.6269], 'angles': [9...",Pm-3m,cubic,-7.257871,47.709746,-0.062245,EuCu,0.071926,0.0000,7.500832,...,One possible application area for this materia...,EuCu,"[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,...","[[1.8134499788284302, -1.8134499788284302, 1.8...",2,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance with the database ID 113701 has ...,"[This material is cubic., This material is not...","[This material is composed of ['Europium', 'Co..."
113702,"{'abc': [7.965815, 7.965816, 7.96581], 'angles...",I-4m2,tetragonal,-7.887140,319.857203,-3.248899,SiO2,0.029099,5.6453,1.559639,...,The material described here is one of the most...,SiO2,"[[0, 7, 0, 5, 0, 6, 0, 8, 0, 1, 0, 2, 0, 4, 0,...","[[-0.06221536546945572, -1.628993272781372, 0....",15,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material represented by this entry in the ...,"[This material is tetragonal., This material i...","[This material is composed of ['Oxygen', 'Sili..."
113703,"{'abc': [7.965159, 7.965154, 7.96516], 'angles...",Fd-3m,cubic,-9.570786,357.328702,-0.206265,Co2Mo4N,0.000000,0.0000,9.584776,...,The material Co2Mo4N with high electrical cond...,Co2Mo4N,"[[0, 22, 0, 21, 0, 17, 0, 2, 0, 3, 0, 1, 0, 10...","[[-0.3580176532268524, 1.3459880352020264, 1.9...",28,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material with the database ID 113703 has a...,"[This material is cubic., This material is not...","[This material is composed of ['Molybdenum', '..."
113704,"{'abc': [13.38737, 16.5407, 19.4082], 'angles'...",Pbcn,orthorhombic,-4.051578,4297.685141,-1.525848,TeAs(Se2F3)2,0.086280,1.4334,3.909250,...,TeAs(Se2F3)2 has interesting electronic and op...,TeAsSe4F6,"[[0, 45, 0, 51, 0, 50, 0, 98, 0, 183, 0, 136, ...","[[-1.1632286310195923, 0.5010343194007874, -2....",192,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance stored in the database has a pre...,"[This material is orthorhombic., This material...","[This material is composed of ['Tellurium', 'F..."
113705,"{'abc': [5.420702, 5.420702, 5.73608], 'angles...",C2/m,monoclinic,-5.235020,165.406146,-1.202585,Pb2O3,0.209392,0.0000,9.284181,...,Pb2O3 is an unstable compound that is not comm...,Pb2O3,"[[0, 7, 0, 4, 0, 8, 0, 9, 0, 6, 0, 5, 0, 3, 0,...","[[-1.6124303340911865, 1.3321336507797241, -0....",10,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material in this database has a pretty for...,"[This material is monoclinic., This material i...","[This material is composed of ['Oxygen', 'Lead..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126330,"{'abc': [5.17955, 5.179548, 15.91706], 'angles...",Cmcm,orthorhombic,-5.416528,238.120897,-0.574610,Fe2O3,1.332748,0.0000,4.454351,...,"Based on the properties described, the materia...",Fe2O3,"[[0, 11, 0, 15, 0, 15, 0, 14, 0, 9, 0, 9, 0, 0...","[[-1.0046907663345337, 0.30613380670547485, 1....",20,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",[0],The substance in question is an orthorhombic m...,"[This material is orthorhombic., This material...","[This material is composed of ['Oxygen', 'Iron..."
126331,"{'abc': [7.898652, 7.898655, 17.29874], 'angle...",C2,monoclinic,-6.171699,691.809324,-1.918218,Bi6P2O15,0.104265,0.0000,7.468825,...,Bi6P2O15 is a type of bismuth-based phosphate ...,Bi6P2O15,"[[0, 28, 0, 32, 0, 24, 0, 42, 0, 42, 0, 25, 0,...","[[-0.07936850190162659, 0.4460204243659973, -2...",46,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material with the formula Bi6P2O15 has a c...,"[This material is monoclinic., This material i...","[This material is composed of ['Bismuth', 'Pho..."
126332,"{'abc': [5.937475,

In [14]:
df['structure_question_list'][113704]

array(['This material is orthorhombic.',
       'This material is not tetragonal.',
       'This material is not hexagonal.', 'This material is not cubic.',
       'This material is not trigonal.',
       'This material is not monoclinic.',
       'This material is not triclinic.'], dtype=object)